# Use SigOpt to tune a Random Forest Classifier in Python
Learn more about our [Python API Client](https://sigopt.com/docs/overview/python).

# Setup
 - Run `pip install sigopt` to download the python API client
 - Run `pip install sklearn` to install [scikit-learn](http://scikit-learn.org), a machine learning library in Python 

Insert your [SigOpt API Token](https://sigopt.com/docs/overview/authentication) below: 

In [ ]:
from sigopt import Connection
conn = Connection(client_token=SIGOPT_API_TOKEN)

# Load dataset
We are using the iris dataset as an example.

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Create an Experiment
Create a SigOpt experiment for the Random Forest parameters.

In [20]:
experiment = conn.experiments().create(
  name="Random Forest (iPython)",
  project="sigopt-examples",
  metrics=[dict(name="cv_accuracies", objective="maximize")],
  parameters=[
    dict(name="max_features", type="int", bounds=dict(min=1, max=X.shape[1])),
    dict(name="n_estimators", type="int", bounds=dict(min=1, max=100)),
    dict(name="min_samples_leaf", type="int", bounds=dict(min=1, max=10))
  ]
)
print("Created experiment: https://sigopt.com/experiment/" + experiment.id)

Created experiment: https://sigopt.com/experiment/118615


# Write evaluate_model Function
Our object metric is the mean of cross validated accuracies, and we use cross validation to prevent overfitting.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import numpy

def evaluate_model(assignments, X, y):
  classifier = RandomForestClassifier(
    n_estimators=assignments['n_estimators'],
    max_features=assignments['max_features'],
    min_samples_leaf=assignments['min_samples_leaf']
  )
  cv_accuracies = cross_val_score(classifier, X, y, cv=5)
  return (numpy.mean(cv_accuracies), numpy.std(cv_accuracies))

# Run the Optimization Loop
Run the [Optimization Loop](https://sigopt.com/docs/overview/optimization) between 10x - 20x the number of parameters.

In [ ]:
for _ in range(60):
    # Receive a Suggestion from SigOpt
    suggestion = conn.experiments(experiment.id).suggestions().create()

    # Evaluate the model locally
    (value, std) = evaluate_model(suggestion.assignments, X, y)

    # Report an Observation (with standard deviation) back to SigOpt
    conn.experiments(experiment.id).observations().create(
        suggestion=suggestion.id,
        value=value,
        value_stddev=std,
    )

# Wrap up the Experiment

In [ ]:
# Re-fetch the best observed value and assignments
best_assignments = conn.experiments(experiment.id).best_assignments().fetch().data[0].assignments
print(best_assignments)

# To wrap up the Experiment, fit the RandomForest on the best assignments
# and train on all available data
rf = RandomForestClassifier(
    n_estimators=best_assignments['n_estimators'],
    max_features=best_assignments['max_features'],
    min_samples_leaf=best_assignments['min_samples_leaf']
)
rf.fit(X, y)